In [1]:
import os

In [2]:
%pwd

'c:\\Users\\mohamed\\Desktop\\stage pfe Orange\\orangePlatform\\orangePlatform\\research'

In [3]:
os.chdir("../")

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/Orange_data.csv")
data.head()

,Period,unknown,3G,2G,LTE
0,2023-03-23,1564.815572,58669.831878,291.658493,1.112119e+06
1,2023-03-24,1485.458404,63449.888426,313.071738,1.173521e+06
2,2023-03-25,1211.967427,65280.190788,326.584583,1.202129e+06
3,2023-03-26,790.808130,67313.354124,337.665937,1.224980e+06
4,2023-03-27,660.593223,66176.025781,324.810713,1.102328e+06


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Period   365 non-null    object 
 1   unknown  365 non-null    float64
 2   3G       365 non-null    float64
 3   2G       365 non-null    float64
 4   LTE      365 non-null    float64
dtypes: float64(4), object(1)
memory usage: 14.4+ KB


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from orangePlatform.constants import *
from orangePlatform.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from orangePlatform import logger

In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [14]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-05-10 18:55:38,574: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-10 18:55:38,576: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-10 18:55:38,578: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-10 18:55:38,584: INFO: common: created directory at: artifacts]
[2024-05-10 18:55:38,586: INFO: common: created directory at: artifacts/data_validation]
